# Quantum Key distribution with and without Eavesdropper

<a id="intro"></a>
# 1. Introduction

**What is key distribution?** Secure communication relies on the ability of the sender (Alice) to encrypt the message in a way that the receiver (Bob) can decrypt it but not an eavesdropper. This security is often accomplished with the use of a **key**, which is a piece of information known only to the sender and receiver and enables them to decrypt and encrypt the message. If a key can be securely distributed between the sender and receiver, the encrypted message can be securely sent over a public channel, since without the key, the probability of successfully decrypting the message is tiny.

Practically, **a key is just a bitstring** - a sequence of 1s and 0s, that is uniquely known only to Alice and Bob, the two communicating parties.

Therefore, the problem of secure communication boils down to secure key distribution. QKD is unique because security against eavesdropping is guaranteed by the laws of quantum mechanics, as opposed to the computational complexity of certain functions, which is what is used in classical key distribution. 

<a id="layout"></a>
## 1.1 Quantum Key Distribution Activity layout

In this lab, and in your homework, we are going to implement the BB84 protocol. Remember from lecture the following steps of the protocol:

1. `SELECT ENCODING`: Alice randomly selects a basis ( × or + ) to encode each bit.
2. `SELECT MEASUREMENT`: Bob randomly selects a basis ( × or + ) to measure each bit.
3. `ENCODE`: Alice creates the quantum states, encoded in the selected bases.
4. `EAVESDROPPER`: Both the cases are shown in the code below, with and without eavesdropper.
5. `SEND`: Alice sends Bob the encoded states, via the quantum channel. 
6. `MEASURE`: Bob measures all the quantum states in his pre-selected measurement bases.
7. `ANNOUNCE BASIS`: Alice announces which basis she used to encode each bit, via the classical channel.
8. `FIND SYMMETRIC KEY`: Alice and Bob discard bits in their key that used a different encoding and decoding basis.

Alice needs to randomly select a bit key which needs to be send to Bob. 
Alice along with Eve and Bob need to randomly select a basis in which to encode each bit of the bit key.

This function takes in the number of bases that Alice needs to randomly pick and returns a list of each chosen encoding represented by either a 0 or a 1

In [1]:
def select_encoding_and_bases(length):
    bitstring = ""
    bases = "" 
    for i in range(length):
        # We use the function getrandbits to get either a 0 or 1 randomly,
        # The "1" in the function argument is the number of bits to be generated
        bitstring += (str(getrandbits(1)))
        # 0 means encode in the (0,1) basis and 1 means encode in the (+,-) basis
        bases += (str(getrandbits(1)))
    
    # return the string of bits and the list of bases they should be encoded in
    return bitstring, bases

Below function takes in the number of bases that Bob and Eve needs to randomly pick and returns a list of each chosen measurement basis represented by either a 0 or a 1

In [2]:
def measure(bases, encoded_qubits, backend):
    bitstring = ''
    for i in range(len(encoded_qubits)):
        qc = encoded_qubits[i]
        
        if bases[i] == "0":
            # 0 means we want to measure in Z basis
            qc.measure(0,0)

        elif bases[i] == "1":
            # 1 means we want to measure in X basis
            qc.h(0)
            qc.measure(0,0)
        
        # Now that the measurements have been added to the circuit, let's run them.
        job = q.execute(qc, backend=backend, shots = 1) # increase shots if running on hardware
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)

        # Append measured bit to Bob's measured bitstring
        bitstring += measured_bit 
        
    return bitstring

Alice and Eve now use their random list of numbers to generate a bunch of quantum states:
    
The table below summarizes the qubit states Alice sends along with Eve after eavesdropping, based on the bit of Alice's `alice_bitstring` the corresponding bit of `selected_bases`:

| Bit in Alice's/Eve's `alice_bitstring/eve_bitstring` | Corresponding bit in `alice_bases/eve_bases` | Encoding basis | Qubit state sent |
|:----------------:|:--------------------------:|:--------------------------:|:---------------:|
| 0 | 0 | $$|0\rangle,|1\rangle$$ |$$|0\rangle$$ |
| 0 | 1 | $$|+\rangle,|-\rangle$$ |$$|+\rangle$$ |
| 1 | 0 | $$|0\rangle,|1\rangle$$ |$$|1\rangle$$ |
| 1 | 1 | $$|+\rangle,|-\rangle$$ |$$|-\rangle$$ |

In [3]:
def encode(bitstring, bases):
    encoded_qubits = []
    for i in range(len(bitstring)):
        # create a brand new quantum circuit called qc. Remember that the qubit will be in state |0> by default
        qc = q.QuantumCircuit(1,1)

        if bases[i] == "0":
            # 0 Means we are encoding in the z basis
            if bitstring[i] == "0":
                # We want to encode a |0> state, as states are intialized
                # in |0> by default we don't need to add anything here
                pass
            
            elif bitstring[i] == "1":
                # We want to encode a |1> state
                # We apply an X gate to generate |1>
                qc.x(0)
                
        elif bases[i] == "1":
            # 1 Means we are encoding in the x basis
            if bitstring[i] == "0":
                # We apply an H gate to generate |+>
                qc.h(0)
            elif bitstring[i] == "1":
                # We apply an X and an H gate to generate |->
                qc.x(0)
                qc.h(0)
            
        # add this quantum circuit to the list of encoded_qubits
        encoded_qubits.append(qc)
        
    return encoded_qubits

Now that Alice has announced what basis she used to encrypt her key, Bob can check against his list and see the places where they matched. The positions where they used the same basis are the places where they will also share the same key value!

In [4]:
def bob_compare_bases(alices_bases, bobs_bases):
    indices = []
    
    for i in range(len(alices_bases)):
        if alices_bases[i] == bobs_bases[i]:
            indices.append(i)
    return indices

#### Bob and Alice know all the positions where they used the same basis to encode and decode a qubit so now if they discard every bit that was encoded using a basis that didn't agree, they will have a shared key!

In [5]:
def construct_key_from_indices(bitstring, indices):
    key = ''
    for idx in indices:
        # For the indices where bases match, the bitstring bit is added to the key
        key = key + bitstring[idx] 
    return key

## QKD Algorithm with no evedropping 

Importing required files

In [6]:
from random import getrandbits
import qiskit as q

Selecting a backend for executing the algorithm

In [7]:
sim_backend = q.Aer.get_backend('qasm_simulator')

Defining the ket length constant, Quantum and Classical channel. This is to simulate the classical and quantum communication in the actual experiment.  

In [8]:
KEY_LENGTH = 500
QUANTUM_CHANNEL = []
CLASSICAL_CHANNEL = []

We can see what alice and bob bit string and bases look like by printing the first 10 elements, this 
should look like a string and a list of random '1's or '0's 

In [9]:
alice_bitstring, alice_bases = select_encoding_and_bases(KEY_LENGTH)
_ , bob_bases = select_encoding_and_bases(KEY_LENGTH)

 
# Preview the first 10 elements of each:
print("Alice randomly generated bitstring: ", alice_bitstring[:10])
print("Alice randomly generated bases: ", alice_bases[:10])

print("Bob randomly generated bases: ", bob_bases[:10])

Alice randomly generated bitstring:  0000010100
Alice randomly generated bases:  0110100110
Bob randomly generated bases:  0010110101


Create Alice's encoded_qubits and send via quantum channel

In [10]:
encoded_qubits = encode(alice_bitstring, alice_bases)
QUANTUM_CHANNEL = encoded_qubits

Measurment of qubits on quantum channel by Bob 

In [11]:
bob_bitstring = measure(bob_bases, QUANTUM_CHANNEL, sim_backend)
print("Bit string at bob's end generated")

Bit string at bob's end generated


Alice sending it bases to Bob via Classical Channel

In [12]:
CLASSICAL_CHANNEL = alice_bases

In [13]:
agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, bob_bases)

In [14]:
CLASSICAL_CHANNEL = agreeing_bases

In [15]:
alice_key = construct_key_from_indices(alice_bitstring, CLASSICAL_CHANNEL)
bob_key = construct_key_from_indices(bob_bitstring, agreeing_bases)

# Preview the first 10 elements of each Key:
print("alice_key: ", alice_key[:10])
print("bob_key:   ", bob_key[:10])
print("Alice's key is equal to Bob's key: ", alice_key == bob_key)
if(not(alice_key == bob_key)):
    print("Thus, this Quantum channel is not safe for data transfer")
elif(alice_key == bob_key):
    print("Thus, this Quantum channel is safe for data transfer")

alice_key:  0000010100
bob_key:    0000010100
Alice's key is equal to Bob's key:  True
Thus, this Quantum channel is safe for data transfer


## QKD Algorithm with evedropping by Eve

Defining the ket length constant, Quantum and Classical channel. This is to simulate the classical and quantum communication in the actual experiment.

In [16]:
KEY_LENGTH = 500
QUANTUM_CHANNEL = []
CLASSICAL_CHANNEL = []

We can see what alice and bob bit string and bases look like by printing the first 10 elements, this should look like a string and a list of random '1's or '0's

In [17]:
alice_bitstring, alice_bases = select_encoding_and_bases(KEY_LENGTH)
_ , bob_bases = select_encoding_and_bases(KEY_LENGTH)
_ , eve_bases = select_encoding_and_bases(KEY_LENGTH)

# Preview the first 10 elements of each:
print("Alice randomly generated bitstring: ", alice_bitstring[:10])
print("Alice randomly generated bases: ", alice_bases[:10])

print("Eve randomly generated bases: ", bob_bases[:10])

print("Bob randomly generated bases: ", eve_bases[:10])

Alice randomly generated bitstring:  1110111001
Alice randomly generated bases:  0010101001
Eve randomly generated bases:  0111111010
Bob randomly generated bases:  1011001110


Create Alice's encoded_qubits and send via quantum channel

In [18]:
encoded_qubits = encode(alice_bitstring, alice_bases)
QUANTUM_CHANNEL = encoded_qubits

Measurment of qubits on quantum channel by Even. Then again encoding the qubits on quantum channel by the randomly selected bases.

In [19]:
eve_bitstring = measure(eve_bases, QUANTUM_CHANNEL, sim_backend)
encoded_qubits = encode(eve_bitstring, eve_bases)
QUANTUM_CHANNEL = encoded_qubits
print("Quantum information is measure and encoded by Eve")

Quantum information is measure and encoded by Eve


Measurment of qubits on quantum channel by Bob

In [20]:
bob_bitstring = measure(bob_bases, QUANTUM_CHANNEL, sim_backend)
print("Bit string at bob's end generated")

Bit string at bob's end generated


Alice sending its bases to Bob via Classical Channel

In [21]:
CLASSICAL_CHANNEL = alice_bases

In [22]:
agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, bob_bases)

In [23]:
CLASSICAL_CHANNEL = agreeing_bases

In [24]:
alice_key = construct_key_from_indices(alice_bitstring, CLASSICAL_CHANNEL)
bob_key = construct_key_from_indices(bob_bitstring, agreeing_bases)

# Preview the first 10 elements of each Key:
print("alice_key: ", alice_key[:10])
print("bob_key: ", bob_key[:10])
print("Alice's key is equal to Bob's key: ", alice_key == bob_key)
if(not(alice_key == bob_key)):
    print("Thus, this Quantum channel is not safe for data transfer")
elif(alice_key == bob_key):
    print("Thus, this Quantum channel is safe for data transfer")

alice_key:  1111000010
bob_key:  1101110010
Alice's key is equal to Bob's key:  False
Thus, this Quantum channel is not safe for data transfer


It can be seen that the when there is evesdroping by a middle person, the channel becomes unsecured. This can be verified by checking the alice and bob keys as shown above.   